In [3]:
import pandas as pd
import geopandas as gpd
import json
from ast import literal_eval
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import json
import base64
import requests
import bs4
from bs4 import BeautifulSoup
import re
import lxml.etree

#pip install geopandas
#pip install spacy
#python -m spacy download en_core_web_sm
#pip install lxml
#pip install parser-libraries 


In [5]:
# read in the dataset 
df = pd.read_json('animal_news_database.json', lines = True)
# Read in the datasets defining our subset data

#undoc = pd.read_csv("undoc_species.csv", sep = ";")
charismatic = pd.read_csv("charismatic_16.csv", sep = ";")
#edge = pd.read_csv("non_charismatic_20.csv", sep = ";")
charismatic = charismatic.rename(columns = {'Scientifc Name' : 'scientific_name'})


# Drop Twitter handles as they are not species names
df = df[df.scientific_name.str.contains('@') == False]
df.reset_index(drop = True, inplace = True)
# How many species listed total in this dataset
#df['scientific_name'].nunique()

# Add new column for storing the values article amount found under each species type 
df['article_amt'] = df['articles'].str.len()




In [6]:
# Read in the CITES data to join with the top 10 (or 20) most reported on species 
cites1 = pd.read_csv("cites1.csv", delimiter = ';')

# Keep only most relevant columns
cites1 = cites1[['Kingdom', 'Class', 'Order', 'Family', 'Genus', 'Species', 'Scientific Name', 'Listing', 'NativeDistributionFullNames', 'Extinct_Distribution']]

cites1 = cites1.rename(columns = {'Scientific Name' : 'scientific_name'})

# Do the same for the entire dataframe
df = df.merge(cites1, on = 'scientific_name', how = 'inner')
df = df.drop(['_id'], axis=1)

# Do the same for the entire dataframe
rizz = df.merge(charismatic, on = 'scientific_name', how = 'inner')
#dg = df.merge(edge, on = 'scientific_name', how = 'inner')


In [8]:
# Explode the articles column
s = rizz['articles'].map(dict.values).explode()
# NaN values need to be droppped
s = s.dropna()
# Drop the column articles and join the exploded values to the dataframe
rizz_articles = rizz.drop(columns=['articles']).join(pd.DataFrame([*s], s.index))


# Processing for articles       
# keep only unique article instances
rizz_media = rizz_articles.drop_duplicates(subset=['text'])

# Transform date column into DateTime
rizz_media["date"] = pd.to_datetime(rizz_media["date"].copy())
# Sort by oldest to newest
rizz_media = rizz_media.sort_values(by="date")


rizz_media = rizz_media.drop(columns=['Unnamed: 8', 'article_amt'])
rizz_media.reset_index(drop = True, inplace = True)
rizz_media = rizz_media[rizz_media['text'].notna()]
rizz_media

/tmp/ipykernel_44/3194386012.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rizz_media["date"] = pd.to_datetime(rizz_media["date"].copy())


,common_name,scientific_name,distribution,iucn_status,date_saved,Kingdom,Class_x,Order,Family,Genus,...,Class_y,Ecosystem,Appendix,"Appendix exceptions, if any",text,title,link,image,date,ner
0,Javan Rhinoceros,Rhinoceros sondaicus,"Bangladesh, Cambodia, China, India, Indonesia,...",Critically Endangered,2020-06-18,Animalia,Mammalia,Perissodactyla,Rhinocerotidae,Rhinoceros,...,Mammal,Terrestrial,1,NaN,"TEMPO.CO, Jakarta - Indonesia is home to two o...",Sumatran Rhino Population Critical,https://news.google.com/articles/CBMiQmh0dHBzO...,https://statik.tempo.co/data/2015/02/09/id_369...,1970-01-01,"[{'sent': 'TEMPO.CO, Jakarta - Indonesia is ho..."
1,Javan Rhinoceros,Rhinoceros sondaicus,"Bangladesh, Cambodia, China, India, Indonesia,...",Critically Endangered,2020-06-18,Animalia,Mammalia,Perissodactyla,Rhinocerotidae,Rhinoceros,...,Mammal,Terrestrial,1,NaN,"TEMPO.CO, Jakarta - Director of the Biodiversi...",Javan Rhinos to be Transferred to Cikepuh Wild...,https://news.google.com/articles/CBMiWmh0dHBzO...,https://statik.tempo.co/data/2011/12/29/id_100...,1970-01-01,"[{'sent': 'TEMPO.CO, Jakarta - Director of the..."
2,Grevy's Zebra,Equus grevyi,"Djibouti, Ethiopia, Kenya, Somalia, South Sudan",EN,2020-06-18,Animalia,Mammalia,Perissodactyla,Equidae,Equus,...,Mammal,Terrestrial,1,NaN,Share\n\n\n\n\n\n\n\n\n\n\n\nA Grevy’s Zebra f...,California zoo welcomes Grevy’s zebra colt foal,https://news.google.com/articles/CBMiSGh0dHBzO...,https://i1.wp.com/www.horsetalk.co.nz/wp-conte...,2001-11-30,"[{'sent': 'A Grevy’s', 'ents': {'ORG': ['Grevy..."
3,Tucuxi,Sotalia fluviatilis,"Brazil, Colombia, Costa Rica, Ecuador, Nicarag...",Data Deficient,2020-06-18,Animalia,Mammalia,Cetacea,Delphinidae,Sotalia,...,Mammal,Aquatic,1,NaN,At the Institute for Marine Mammal Studies in ...,Why dolphins are deep thinkers,https://news.google.com/articles/CBMiQGh0dHBzO...,https://i.guim.co.uk/img/static/sys-images/Env...,2003-07-03,[{'sent': 'At the Institute for Marine Mammal ...
4,Western Gorilla,Gorilla gorilla,"Angola, Cameroon, Central African Republic, Co...",CR,2020-06-18,Animalia,Mammalia,Primates,Hominidae,Gorilla,...,Mammal,Terrestrial,1,NaN,By Helen Briggs\n\nBBC News Online science rep...,Gorilla study gives social clues,https://news.google.com/articles/CBMiNWh0dHA6L...,http://newsimg.bbc.co.uk/media/images/39953000...,2004-03-23,"[{'sent': 'By Helen Briggs ', 'ents': {'PERSO..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2844,Irrawaddy Dolphin,Orcaella brevirostris,"Australia, Bangladesh, Brunei Darussalam, Camb...",EN,2020-06-18,Animalia,Mammalia,Cetacea,Delphinidae,Orcaella,...,Mammal,Aquatic,1,NaN,The Campaign to Ban Trophy Hunting’s (CBTH) fo...,A new book could be the final nail in the coff...,https://news.google.com/articles/CAIiEGKvqlzhh...,https://www.thecanary.co/wp-content/uploads/20...,2020-05-30,[{'sent': '(CBTH) founder Eduardo Gonçalves ha...
2845,Fin Whale,Balaenoptera physalus,"Algeria, Angola, Antarctica, Argentina, Austra...",VU,2020-06-18,Animalia,Mammalia,Cetacea,Balaenopteridae,Balaenoptera,...,Mammal,Aquatic,1,NaN,A 12-metre-long fin whale which washed up at H...,Fin whale washed up on Essex beach has been re...,https://news.google.com/articles/CBMiUmh0dHBzO...,https://www.scubadivermag.com/wp-content/uploa...,2020-05-30,[{'sent': 'A 12-metre-long fin whale which was...
2846,Irrawaddy Dolphin,Orcaella brevirostris,"Australia, Bangladesh, Brunei Darussalam, Camb...",EN,2020-06-18,Animalia,Mammalia,Cetacea,Delphinidae,Orcaella,...,Mammal,Aquatic,1,NaN,"In a unique initiative, researchers from the I...",IIT researchers help Chilika Lake in Odisha to...,https://news.google.com/articles/CBMiiwFodHRwc...,https://www.deccanherald.com/sites/dh/files/ar...,2020-05-31,"[{'sent': 'In a unique initiative, researchers..."
2847,Panther,Panthera pardus,"Afghanistan, Algeria, Angola, Armenia, Azerbai...",VU,2020-06-18,Animalia,Mammalia,Carnivora,Felidae,Panthera,...,Mammal,Terrestrial,1,NaN,Lonzo

In [9]:
import requests

def get_full_url(short_url):
        try:
            session = requests.Session()  # so connections are recycled
            resp = session.head(short_url, allow_redirects=True, timeout=10)
            url = resp.url
            
            if 'url=' in url and '&ct' in url:
                return url.split('url=')[1].split('&ct')[0]
            else:
                return short_url
               
        except Exception:   
            return short_url

In [10]:
for link in rizz_media['link']:
    url = link
    browser = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers = {'User-Agent':browser,}
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    links = soup.findAll("a")
    
    print(soup)
    
    
    break

    l = []

    for x in soup.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):
        l.append(re.split(":(?=http)",link["href"].replace("/url?q=",""))[0])

print(l)

<!DOCTYPE html>
<html dir="ltr" lang="en-US"><head><style nonce="bvIZafA0grjnkM1REZevQA">
a, a:link, a:visited, a:active, a:hover {
  color: #1a73e8;
  text-decoration: none;
}
body {
  font-family: Roboto,RobotoDraft,Helvetica,Arial,sans-serif;
  text-align: center;
  -ms-text-size-adjust: 100%;
  -moz-text-size-adjust: 100%;
  -webkit-text-size-adjust: 100%;
}
.box {
  border: 1px solid #dadce0;
  box-sizing: border-box;
  border-radius: 8px;
  margin: 24px auto 5px auto;
  max-width: 800px;
  padding: 24px;
}
h1 {
  color: #2c2c2c;
  font-size: 24px;
  hyphens: auto;
  margin: 24px 0;
}
.icaCallout {
  background-color: #f8f9fa;
  padding: 12px 16px;
  border-radius: 10px;
  margin-bottom: 10px;
}
p, .sub, .contentText, .icaCallout {
  color: #5f6368;;
  font-size: 14px;
  line-height: 20px;
  letter-spacing: 0.2px;
  text-align: left;
}
.signin {
  text-align: right;
}
.saveButtonContainer,
.saveButtonContainerNarrowScreen {
  width: 100%;
  margin-top: 12px;
}
.customButtonContain

NameError: name 'l' is not defined

In [11]:
for link in rizz_media['link']:
    session = requests.Session()  # so connections are recycled
    resp = session.head(link, allow_redirects=True, timeout=10)
    url = resp.url
    
    print(url)
    break

https://consent.google.com/ml?continue=https://news.google.com/articles/CBMiQmh0dHBzOi8vZW4udGVtcG8uY28vcmVhZC84MDY0NDQvc3VtYXRyYW4tcmhpbm8tcG9wdWxhdGlvbi1jcml0aWNhbNIBQmh0dHBzOi8vd3d3LnRlbXBvLmNvL2FtcC84MDY0NDQvc3VtYXRyYW4tcmhpbm8tcG9wdWxhdGlvbi1jcml0aWNhbA?hl%3Den-US%26gl%3DUS%26ceid%3DUS:en&gl=FI&hl=en-US&pc=n&src=1


In [ ]:
## Decoding the google news article links with base64 so they can be utilized by other APIs

rizz_media['url'] = None
test = []

for link in rizz_media['link']:
    coded = link.strip("https://news.google.com/articles/")
    
    url = base64.b64decode(coded + '==')[4:].decode('utf-8', "backslashreplace").split('\\')[0]
    #dic = {'url': [url]}
    #rizz_media.append(pd.DataFrame(dic))
    test.append(url)
    #test = pd.concat([rizz_media, pd.DataFrame(dic)])
   

In [ ]:
for i in test:
    for x in i.values():
        for y in x:
            print(y)
            print(y[y.find(b'https'):])
            
            
            
            break

In [ ]:
rizz_media['url'] = pd.Series(test)
rizz_media

In [ ]:
from requests.structures import CaseInsensitiveDict

sharecount = []
for url in test[:20]: 
    
    api = "https://api.sharedcount.com/v1.1?apikey=2a0336186dc8f5740fef9c1eb81947d66119c29f&url=" + url
    
    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"
    
    resp = requests.get(api, headers=headers)
    details = resp.json()
    for n in range(len(details)):
        count = details[n]['Facebook']['share_count']
        
    
    #print(resp)